<a href="https://colab.research.google.com/github/HSS107048212/hoop4s/blob/main/%E5%90%88%E4%BD%9C%E7%A4%BE%E7%95%B6%E6%97%A5%E3%80%8C%E5%8F%96%E8%B2%A8%E9%80%9A%E7%9F%A5%E3%80%8D%E5%AF%84%E9%80%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-docx
# Installation and imports
!pip3 install pygsheets
!pip3 install gspread-dataframe==3.2.1
!pip3 install openpyxl
# Install required package for integrating Pandas with Google Sheets
!pip install gspread-dataframe


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha256=bf6e63620092820968c26a150c73ea873ef7f53c93fd889d070c385810849e2e
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: gspread-dataframe
    Found existing installation: gspread-dataframe 3.3.1
    Uninstalling gspread-dataframe-3.3.1:
      Successfully uninstalled gspread-dataframe-3.3.1


# SERVER端程式

In [ ]:
import os
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

import pandas as pd
import pygsheets
import gspread
import re
from docx import Document
import smtplib

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
#from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Initializations
#HOME_PATH = '/home/iss/realplus/'
#HOME_PATH = ''
HOME_PATH = '/content/' # path of Colab for testing
# Initialize the pygsheets authorization
gc = pygsheets.authorize(service_file=f'{HOME_PATH}orderdata-copyto-parttime-5c19ba8fcba4.json')  # Gild學長後續請將.json金鑰設定在local路徑


# Google sheet URL and authentication
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1EDtLvs_Rz_bwsew0BfZSKHBZpp5ZNDSKg3gLv_XZCbU/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("可寄發取貨通知")

# Convert worksheet data to DataFrame
Order_data = pd.DataFrame(wks.get_all_records())

# Google sheet URL and authentication
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1pSHCFR-pHePTHDCMlnkS5vPS6imU_fLBk_r9eYmBuV0/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("工作表1")

# Convert worksheet data to DataFrame
Price_data = pd.DataFrame(wks.get_all_records())

# 假設Price_data是一個Pandas DataFrame，銷售價格列是 "銷售價格"。
Price_data["銷售價格"] = Price_data["銷售價格"].astype(str)  # 將列轉換為字符串

# 使用正則表達式去除非數字字符
Price_data["銷售價格"] = Price_data["銷售價格"].apply(lambda x: re.sub(r'\D', '', x)).astype(int)

Price_data["商品全稱"] = Price_data["供應商"] + Price_data["商品名稱"]



# Load data
product_names = Price_data['商品全稱']

### Calculate Subtotal: Add up the quantity ordered for each item on the day
for index, row in Order_data.iterrows():
    subtotal = 0
    for product in product_names:
        quantity = row.get(product, 0)
        quantity = 0 if quantity == '' else float(quantity)
        price = float(Price_data.loc[Price_data['商品全稱'] == product, '銷售價格'].values[0])
        subtotal += quantity * price
    Order_data.at[index, '小計'] = subtotal


# 設定寄件者的電子郵件地址和密碼作為環境變數
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")

if not sender_email or not sender_password:
    print("Sender email and password not provided in environment variables.")
    exit(1)

# Define the output directory based on the current date
today = datetime.now().strftime("%Y-%m-%d")
# Create a directory to save the individual Excel files
output_directory = f'{HOME_PATH}content/{today}/' # gild學長在麻煩您設定成為server的路徑
import os
os.makedirs(output_directory, exist_ok=True)
# Your loop for sending emails and creating Word documents
for index, row in Order_data.iterrows():
    recipient_name = row["訂購者姓名"]
    order_num = row["訂單編號"]
    product_quantities = row[6:-2]
    total_amount = row["小計"]

    # Create email content
    email_content = f"Dear {recipient_name},\n十里方圓合作社提醒您，已經備妥您訂購的品項，請確認:\n\n"
    email_content += f"===================================\n"
    email_content += f"系統自動生成的訂單編號: {order_num} \n"
    email_content += f"===================================\n"

    # Define sale_price or retrieve it from Price_data
    sale_price = 0  # Initialize with a default value
    for product, quantity in product_quantities.items():
        if quantity != "":
            # Check if the product is in the "商品名稱" column in Price_data and set sale_price accordingly
            if product in Price_data["商品全稱"].values:
                sale_price = Price_data.loc[Price_data["商品全稱"] == product, "銷售價格"].values[0]
            email_content += f"{product}: {quantity} 件，該商品單價是{sale_price}元\n"

    email_content += f"===================================\n"
    email_content += f"Total amount: {total_amount} 新台幣\n\n"
    email_content += f"===================================\n"
    email_content += f"請記得攜帶手機(確認訂單編號)與「提袋(環保3Q)」\n"
    email_content += f"取貨時間請參考：https://www.coop4sustainability.live/%E6%B0%B4%E6%9C%A8%E5%8A%87%E9%BB%9E \n"
    email_content += f"取貨地點：新竹市東區光復路二段101號水木餐廳十里方圓櫃位 https://goo.gl/maps/c9UXWnfc1CfC3jGeA \n"
    email_content += f"===================================\n"
    email_content += f"溫馨提示：社員您好，請根據取貨通知訂單的金額付款，待完成付款，則此次訂單成立，因此將累積相應的t-point點數到您的賬戶，t-point點數將作為合作社年度結餘分配之依據。（t-point點數與新台幣兌換比例為1：1）\n"

    # Create a Word document for each email
    doc = Document()
    doc.add_heading(f"Order for {recipient_name}", level=1)
    doc.add_paragraph(email_content)

    # Save the Word document
    output_filename = os.path.join(output_directory, f"Order_{order_num}.docx")
    doc.save(output_filename)

    # Email settings
    sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
    sender_password = "jwsscmoiffdyukmm"
    recipient_email = str(row["電子郵件地址"])  # 確保將電子郵件地址轉換為字符串

    subject = "【十里方圓合作社】取貨通知"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(email_content, "plain"))

    try:
        # Connect to SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587) #需注意中間填寫是gmail
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, message.as_string())

        # Close the SMTP server
        server.quit()

        print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
    except Exception as e:
        print(f"Failed to send email to {recipient_name}. Error: {str(e)}")

# Final log message
print(f"All emails saved as Word documents in {output_directory}")

# Get list of files in the output directory
files = os.listdir(output_directory)


# Load your service account key JSON file
credentials = service_account.Credentials.from_service_account_file(
    f'{HOME_PATH}orderdata-copyto-parttime-bbb4e17511c7.json',
    scopes=['https://www.googleapis.com/auth/drive']  # Gild 學長, please set the JSON path
)

# Create Google Drive API client
drive_service = build('drive', 'v3', credentials=credentials)

# Specify the target folder ID (obtained from the URL you provided)
folder_id = '1qZSvLUY6AzbesmrYh2pF2Bw47JtArH_o'

for file_name in files:
    # Specify the path of the file to upload
    file_path = f'{HOME_PATH}content/{today}/{file_name}'  # Gild 學長, please modify the path

    # Create a file media upload object
    media = MediaFileUpload(file_path, mimetype='application/octet-stream')

    # Upload the file to the target folder
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }

    uploaded_file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()

    print(f'文件 "{file_name}" 已成功上傳到 Google Drive 文件夾，文件 ID：{uploaded_file.get("id")}')


Email sent to 陳皆錫, recipient's email address: ubo11185@gapp.nthu.edu.tw
Email sent to 吳鋒, recipient's email address: wufengcn001@gmail.com
Email sent to 吳鋒, recipient's email address: wufengcn001@gmail.com
All emails saved as Word documents in content/2023-09-27/
文件 "Order_20230915001.docx" 已成功上傳到 Google Drive 文件夾，文件 ID：16PdqadezNNRAFJAS96pUzUqPoimLJFIM
文件 "Order_20230915002.docx" 已成功上傳到 Google Drive 文件夾，文件 ID：1QjWKSCaY-yPQCusTqKKSih2bYdHhC4e6
文件 "Order_20230915003.docx" 已成功上傳到 Google Drive 文件夾，文件 ID：1NrQiEi2tme6WAo4zML4yalXTNRA_L0HC


# 原始程式

In [ ]:
!pip3 install pygsheets
!pip3 install gspread-dataframe==3.2.1

In [ ]:
import pygsheets
gc = pygsheets.authorize(service_file='/content/orderdata-copyto-parttime-5c19ba8fcba4.json') # Gild學長後續請將.json金鑰設定在local路徑
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd

HOME_PATH = ''

# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1EDtLvs_Rz_bwsew0BfZSKHBZpp5ZNDSKg3gLv_XZCbU/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("可寄發取貨通知")
#讀取成 df
Order_data = pd.DataFrame(wks.get_all_records())

# google sheet 網址與認證
SPREADSHEET = 'https://docs.google.com/spreadsheets/d/1pSHCFR-pHePTHDCMlnkS5vPS6imU_fLBk_r9eYmBuV0/edit?usp=sharing'
sht = gc.open_by_url(SPREADSHEET)
wks = sht.worksheet_by_title("工作表1")
#讀取成 df
Price_data = pd.DataFrame(wks.get_all_records())

import re
# Assuming pricedata is a DataFrame with your data
prices = Price_data["銷售價格"].apply(lambda x: re.sub(r'\D', '', x)).astype(int)
Price_data["銷售價格"] = prices
Price_data

Price_data["商品全稱"] = Price_data["供應商"] + Price_data["商品名稱"]
Price_data

### 加總：各品項當日訂購數量

# 获取 Price_data 中的商品全称列
product_names = Price_data['商品全稱']

# 遍历 Order_data 中的每一行
for index, row in Order_data.iterrows():
    subtotal = 0  # 初始化小计为0
    for product in product_names:
        quantity = row.get(product, 0)  # 获取数量，如果没有则默认为0
        if quantity == '':
            quantity = 0
        else:
            quantity = float(quantity)  # 强制转换为浮点数
        price = float(Price_data.loc[Price_data['商品全稱'] == product, '銷售價格'].values[0])  # 强制转换为浮点数
        subtotal += quantity * price
    Order_data.at[index, '小計'] = subtotal

# 根据电子邮件地址向每个订购者发送帐单
for index, row in Order_data.iterrows():
    email = row['電子郵件地址']
    name = row['訂購者姓名']
    subtotal = row['小計']
    # 此处添加发送帐单的代码，可以将帐单信息发送到对应的电子邮件地址

# 打印 Order_data，包括小计
Order_data

import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os
from docx import Document
from datetime import datetime


# 設定寄件者的電子郵件地址和密碼作為環境變數
os.environ["COOP_EMAIL"] = "coop4s.hsinchu@gmail.com"
os.environ["COOP_PASSWORD"] = "coop4s@nthu"

# Get sender email and password from environment variables
sender_email = os.getenv("COOP_EMAIL")
sender_password = os.getenv("COOP_PASSWORD")

if not sender_email or not sender_password:
    print("Sender email and password not provided in environment variables.")
    exit(1)

# Define the output directory based on the current date
today = datetime.now().strftime("%Y-%m-%d")
# Create a directory to save the individual Excel files
output_directory = f'{HOME_PATH}content/{today}/' # gild學長在麻煩您設定成為server的路徑
import os
os.makedirs(output_directory, exist_ok=True)
# Your loop for sending emails and creating Word documents
for index, row in Order_data.iterrows():
    recipient_name = row["訂購者姓名"]
    order_num = row["訂單編號"]
    product_quantities = row[6:-2]
    total_amount = row["小計"]

    # Create email content
    email_content = f"Dear {recipient_name},\n十里方圓合作社提醒您，已經備妥您訂購的品項，請確認:\n\n"
    email_content += f"===================================\n"
    email_content += f"系統自動生成的訂單編號: {order_num} \n"
    email_content += f"===================================\n"

    # Define sale_price or retrieve it from Price_data
    sale_price = 0  # Initialize with a default value
    for product, quantity in product_quantities.items():
        if quantity != "":
            # Check if the product is in the "商品名稱" column in Price_data and set sale_price accordingly
            if product in Price_data["商品全稱"].values:
                sale_price = Price_data.loc[Price_data["商品全稱"] == product, "銷售價格"].values[0]
            email_content += f"{product}: {quantity} 件，該商品單價是{sale_price}元\n"

    email_content += f"===================================\n"
    email_content += f"Total amount: {total_amount} 新台幣\n\n"
    email_content += f"===================================\n"
    email_content += f"請記得攜帶手機(確認訂單編號)與「提袋(環保3Q)」\n"
    email_content += f"取貨時間請參考：https://www.coop4sustainability.live/%E6%B0%B4%E6%9C%A8%E5%8A%87%E9%BB%9E \n"
    email_content += f"取貨地點：新竹市東區光復路二段101號水木餐廳十里方圓櫃位 https://goo.gl/maps/c9UXWnfc1CfC3jGeA \n"
    email_content += f"===================================\n"
    email_content += f"溫馨提示：社員您好，請根據取貨通知訂單的金額付款，待完成付款，則此次訂單成立，因此將累積相應的t-point點數到您的賬戶，t-point點數將作為合作社年度結餘分配之依據。（t-point點數與新台幣兌換比例為1：1）\n"

    # Create a Word document for each email
    doc = Document()
    doc.add_heading(f"Order for {recipient_name}", level=1)
    doc.add_paragraph(email_content)

    # Save the Word document
    output_filename = os.path.join(output_directory, f"Order_{order_num}.docx")
    doc.save(output_filename)

    # Email settings
    sender_email = os.getenv("COOP_EMAIL")  # 使用正確的環境變數名稱
    sender_password = "jwsscmoiffdyukmm"
    recipient_email = str(row["電子郵件地址"])  # 確保將電子郵件地址轉換為字符串

    subject = "【十里方圓合作社】取貨通知"
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = recipient_email
    message["Subject"] = subject
    message.attach(MIMEText(email_content, "plain"))

    try:
        # Connect to SMTP server
        server = smtplib.SMTP("smtp.gmail.com", 587) #需注意中間填寫是gmail
        server.starttls()
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, message.as_string())

        # Close the SMTP server
        server.quit()

        print(f"Email sent to {recipient_name}, recipient's email address: {recipient_email}")
    except Exception as e:
        print(f"Failed to send email to {recipient_name}. Error: {str(e)}")


print(f"All emails saved as Word documents in {output_directory}")

files = os.listdir(output_directory)

import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# 載入你的服務帳戶金鑰 JSON 檔案
credentials = service_account.Credentials.from_service_account_file(
    f'{HOME_PATH}orderdata-copyto-parttime-bbb4e17511c7.json', scopes=['https://www.googleapis.com/auth/drive'] # Gild學長請設定JSON路徑
)

# 建立 Google Drive API 客戶端
drive_service = build('drive', 'v3', credentials=credentials)



# 指定目標文件夾的ID（從你提供的URL中獲得）
folder_id = '1qZSvLUY6AzbesmrYh2pF2Bw47JtArH_o'



for file_name in files:
  # 指定要上傳的文件路徑
  file_path = f'{HOME_PATH}content/{today}/{file_name}'  #Gild學長麻煩修改路徑
  # 創建文件媒體上傳物件
  media = MediaFileUpload(file_path, mimetype='application/octet-stream')
# 上傳文件到目標文件夾
  file_metadata = {
      'name': file_name,
      'parents': [folder_id]
  }

  uploaded_file = drive_service.files().create(
      body=file_metadata,
      media_body=media,
      fields='id'
  ).execute()

  print(f'文件 "{file_name}" 已成功上傳到 Google Drive 文件夾，文件 ID：{uploaded_file.get("id")}')



Email sent to 陳皆錫, recipient's email address: ubo11185@gapp.nthu.edu.tw
Email sent to 吳鋒, recipient's email address: wufengcn001@gmail.com
Email sent to 吳鋒, recipient's email address: wufengcn001@gmail.com
All emails saved as Word documents in content/2023-09-19/
文件 "Order_20230915001.docx" 已成功上傳到 Google Drive 文件夾，文件 ID：1Nf8M3mIP7wWakEyGb-jdo05rDydo_Zxl
文件 "Order_20230915002.docx" 已成功上傳到 Google Drive 文件夾，文件 ID：1yd4QQ51fJpDBAGy9e5nnvhyuZhi-y1Ng
文件 "Order_20230915003.docx" 已成功上傳到 Google Drive 文件夾，文件 ID：1HR_Y06Ch2nu9tR1dUIqHLb5YSG2hNvns
